# COIN Project

Authors:

    - Mona
    - Roman
    - Nick
    - Mateo

# 1. Setup

### Packages

In [176]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import tweepy
#import dotenv
import os
import yaml
import datetime
from pprint import pprint
import time
import re


### Environment Variables

In [148]:
#dotenv.load_dotenv()

In [149]:
with open("../twitter.yml", "r") as f:
    config = yaml.safe_load(f)

In [150]:
config

{'accounts': ['Fridays4future',
  'FridayForFuture',
  'GretaThunberg',
  'Luisamneubauer',
  'AnnaUKSCN',
  'AnunaDe',
  'adelaidecharli2',
  'HollyWildChild',
  'AlexandriaV2005',
  'fff_europe',
  'fff_digital',
  'FFF_Sweden',
  'FFF_Scotland',
  'FFF_Berlin',
  'FFFireland',
  'fff_hamburg'],
 'hashtags': {'green': ['EndCoal',
   'EndFossilFuels',
   'PeopleNotProfit',
   'NoMoreEmptyPromises',
   'UprootTheSystem',
   'FridaysForFuture',
   'ClimateAction',
   'ClimateJustice',
   'ClimateEmergency',
   'ClimateStrike',
   'SaveThePlanet'],
  'brown': ['climatescam',
   'climatechangehoax',
   'fakeclimate',
   'climatehoax',
   'globalwarmingisahoax'],
  'neutral': ['ClimateCrisis', 'ClimateChange', 'Climate', 'GlobalWarming']}}

### Database Connection

In [151]:
#TBD

### Twitter API Connection

In [165]:
#Research Access:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAGjtbgEAAAAAyx9oZIai1hXZ9OyDhUUFMZQivpc%3DUunewUXR9hw3nyKQhjqdmfg7zSAoa1nPv6WKLLSPB7OwKwYBP3'

#Initializing the client to request the Twitter API
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

#expansions = ['attachments.poll_ids', 'attachments.media_keys', 'author_id', 'entities.mentions.username', 'geo.place_id', 'in_reply_to_user_id', 'referenced_tweets.id', 'referenced_tweets.id.author_id']
expansions = ['attachments.media_keys','author_id','geo.place_id','referenced_tweets.id', 'entities.mentions.username','referenced_tweets.id.author_id']
max_results = 100
media_fields = ["duration_ms", "height", "media_key", "preview_image_url", "type", "url", "width", "public_metrics", "non_public_metrics", "organic_metrics", "promoted_metrics", "alt_text"]
#next_token = ''#(str | None) – This parameter is used to get the next ‘page’ of results. The value used with the parameter is pulled directly from the response provided by the API, and should not be modified. You can learn more by visiting our page on pagination.
place_fields = ['country', 'country_code', 'full_name', 'geo', 'id', 'name', 'place_type']
#poll_fields #(list[str] | str | None) – poll_fields
#since_id #(int | str | None) – Returns results with a Tweet ID greater than (for example, more recent than) the specified ID. The ID specified is exclusive and responses will not include it. If included with the same request as a start_time parameter, only since_id will be used.
sort_order = 'recency'
start_time = '2017-01-01T00:00:00Z' #(datetime.datetime | str | None) – YYYY-MM-DDTHH:mm:ssZ (ISO 8601/RFC 3339). The oldest UTC timestamp from which the Tweets will be provided. Timestamp is in second granularity and is inclusive (for example, 12:00:01 includes the first second of the minute). By default, a request will return Tweets from up to 30 days ago if you do not include this parameter.
#end_time = '2019-07-15T23:00:00Z'
#tweet_fields = ["attachments", "author_id", "context_annotations", "conversation_id", "created_at", "entities", "geo", "id", "in_reply_to_user_id", "lang", "non_public_metrics", "public_metrics", "organic_metrics", "promoted_metrics", "possibly_sensitive", "referenced_tweets", "reply_settings", "source", "text", "withheld"]
tweet_fields = ["author_id","conversation_id","context_annotations","created_at","geo","public_metrics"]
#until_id #(int | str | None) – Returns results with a Tweet ID less than (that is, older than) the specified ID. Used with since_id. The ID specified is exclusive and responses will not include it.
#user_fields can be extracted from the .includes attribute of the tweepy response
user_fields = ["created_at", "description", "location", "name", "pinned_tweet_id", "profile_image_url", "protected", "public_metrics", "url", "username", "verified", "withheld"]


# 2. Load and Transform Data

### 2.1 Load

In [166]:
#specify tweets to look up

#query_list = ['from:Luisamneubauer -is:retweet']

#query_list = ['FridaysForFuture -is:retweet']


''''query_list = ['from:Fridays4future -is:retweet',
              'from:FridayForFuture -is:retweet',
              'from:GretaThunberg -is:retweet',
              'from:Luisamneubauer -is:retweet',
              'from:AnnaUKSCN -is:retweet',
              'from:AnunaDe -is:retweet',
              'from:adelaidecharli2 -is:retweet',
              'from:HollyWildChild -is:retweet',
              'from:AlexandriaV2005 -is:retweet',
              'from:fff_europe -is:retweet',
              'from:fff_digital -is:retweet',
              'from:FFF_Sweden -is:retweet',
              'from:FFF_Scotland -is:retweet',
              'from:FFF_Berlin -is:retweet',
              'from:FFFireland -is:retweet',
              'from:fff_hamburg -is:retweet'''''

query_list = ['FridaysForFuture -is:retweet',
              'SchoolStrike4Climate -is:retweet',
              'fridays4future -is:retweet']





In [167]:
def search_all_tweets(query):
    response = client.search_all_tweets(
                               query=query
                              ,expansions=expansions
                              ,max_results=max_results
                              ,media_fields=media_fields
                              ,place_fields=place_fields
                              ,sort_order=sort_order
                              ,start_time=start_time
                              #,end_time=end_time
                              ,tweet_fields=tweet_fields
                              ,user_fields=user_fields
                            )
    return response

In [168]:
def search_all_tweets_with_next_token(query, next_token):
    response = client.search_all_tweets(
                               query=query
                              ,expansions=expansions
                              ,max_results=max_results
                              ,media_fields=media_fields
                              ,place_fields=place_fields
                              ,sort_order=sort_order
                              ,start_time=start_time
                              #,end_time=end_time
                              ,tweet_fields=tweet_fields
                              ,user_fields=user_fields
                              ,next_token=next_token
                            )
    return response

In [169]:
def fetch_all_data(query, iteration_count, next_token):

    """ Fetches iteration_count * 100 Tweets from Twitter API using the set query params, including the initial next_token
    
    Parameters:
    ------------
    * iteration_count     : Integer indicating the max number of requests 
    * next_token          : String containing the next token for pagination
    
    """
    
    # Initializing the return parameters
    tweets = []
    users = []
    ref_tweets = []
    media = []
    places = []
    
    for i in range(0, iteration_count):

        if(len(next_token) == 0):
            response = search_all_tweets(query)
        else:
            response = search_all_tweets_with_next_token(query, next_token)

        tweets.append(response.data)
        users.append(response.includes['users'])
        ref_tweets.append(response.includes['tweets'])
        if "media" in response.includes.keys():
            media.append(response.includes['media'])
        
        if('places' in response.includes.keys()):
            places.append(response.includes['places'])
            
        try:
            next_token = response.meta['next_token']
        except:
            print('No more Tweets to fetch')
            break


    # Logging the last next_token for pagination into console --> needed for later requesting of the next tweets
    if('next_token' in response.meta):
        print(response.meta['next_token'])
        
    return tweets, users, ref_tweets, media, places

In [170]:
def unpack_dict(unpack_dict):
    unpacked = [v for k, v in unpack_dict.items()]
    return unpacked
def unpack_attachment(attachment_dict):
    return "|".join(attachment_dict["media_keys"]) if "media_keys" in attachment_dict.keys() else np.nan
def unpack_geo(geo_dict):
    return geo_dict["type"], "|".join([str(x) for x in geo_dict["bbox"]])
def unpack_geo_2(geo_dict):
    return geo_dict["place_id"]

In [171]:
def query_tweets(query):    
    tweets, users, ref_tweets, media, places = fetch_all_data(query, 100000, '')

    user_df = pd.DataFrame(data=[user for sublist in users for user in sublist])
    tweet_df = pd.DataFrame(data=[tweet for sublist in tweets for tweet in sublist])
    ref_tweet_df = pd.DataFrame(data=[tweet for sublist in ref_tweets for tweet in sublist])
    media_df = pd.DataFrame(data=[item for sublist in media for item in sublist])
    place_df = pd.DataFrame(data=[place for sublist in places for place in sublist])
    
    user_df[["followers_count", "following_count", "tweet_count", "listed_count"]] = user_df.apply(lambda x: unpack_dict(x["public_metrics"]), axis=1, result_type="expand")
    del user_df["public_metrics"]

    tweet_df[["retweet_count", "reply_count", "like_count", "quote_count"]] = tweet_df.apply(lambda x: unpack_dict(x["public_metrics"]), axis=1, result_type="expand")
    tweet_df["media_keys"] = tweet_df.loc[tweet_df["attachments"].notna()].apply(lambda x: unpack_attachment(x["attachments"]), axis=1, result_type="expand")
    del tweet_df["public_metrics"]
    del tweet_df["attachments"]
    del tweet_df["referenced_tweets"]

    ref_tweet_df[['retweet_count', 'reply_count', 'like_count', 'quote_count']] = ref_tweet_df.apply(lambda x: unpack_dict(x["public_metrics"]), axis=1, result_type="expand")
    ref_tweet_df["media_keys"] = ref_tweet_df.loc[ref_tweet_df["attachments"].notna()].apply(lambda x: unpack_attachment(x["attachments"]), axis=1, result_type="expand")
    #ref_tweet_df['place_id'] = ref_tweet_df.loc[ref_tweet_df["geo"].notna()].apply(lambda x: unpack_geo_2(x["geo"]), axis=1, result_type="expand")
    del ref_tweet_df["public_metrics"]
    del ref_tweet_df["attachments"]
    #del ref_tweet_df["geo"]
    del ref_tweet_df["referenced_tweets"]
    
    '''if "geo" in place_df.columns:
        place_df[["type", "coordinates"]] = place_df.loc[place_df["geo"].notna()].apply(lambda x: unpack_geo(x["geo"]), axis=1, result_type="expand")
        del place_df["geo"]'''
        
    tweet_df.text = tweet_df.text.apply(lambda x: re.sub(r"[^A-Za-z0-9\w\s:@,]", "", x))
    return user_df, tweet_df, ref_tweet_df, place_df, media_df

In [172]:
all_user = all_tweets = all_ref_tweets = all_places = all_media = pd.DataFrame()
for i in query_list:
    print(i)
    
    try:
            user_df, tweet_df, ref_tweet_df, place_df, media_df = query_tweets(i)
            all_user = pd.concat([all_user, user_df])
            all_tweets = pd.concat([all_tweets, tweet_df])
            all_ref_tweets = pd.concat([all_ref_tweets, ref_tweet_df])
            all_places = pd.concat([all_places, place_df])
            all_media = pd.concat([all_media, media_df])
    except:
            print('Return')
            pass
            
    
    


FridaysForFuture -is:retweet


Rate limit exceeded. Sleeping for 395 seconds.
Rate limit exceeded. Sleeping for 665 seconds.
Rate limit exceeded. Sleeping for 900 seconds.
Rate limit exceeded. Sleeping for 899 seconds.
Rate limit exceeded. Sleeping for 515 seconds.
Rate limit exceeded. Sleeping for 718 seconds.
Rate limit exceeded. Sleeping for 533 seconds.
Rate limit exceeded. Sleeping for 565 seconds.
Rate limit exceeded. Sleeping for 838 seconds.
Rate limit exceeded. Sleeping for 797 seconds.


Return
SchoolStrike4Climate -is:retweet


Rate limit exceeded. Sleeping for 530 seconds.
Rate limit exceeded. Sleeping for 689 seconds.
Rate limit exceeded. Sleeping for 525 seconds.
Rate limit exceeded. Sleeping for 858 seconds.
Rate limit exceeded. Sleeping for 720 seconds.
Rate limit exceeded. Sleeping for 871 seconds.
Rate limit exceeded. Sleeping for 820 seconds.


No more Tweets to fetch
fridays4future -is:retweet


Rate limit exceeded. Sleeping for 746 seconds.
Rate limit exceeded. Sleeping for 441 seconds.
Rate limit exceeded. Sleeping for 444 seconds.
Rate limit exceeded. Sleeping for 831 seconds.
Rate limit exceeded. Sleeping for 467 seconds.
Rate limit exceeded. Sleeping for 461 seconds.
Rate limit exceeded. Sleeping for 459 seconds.


Return


In [173]:
all_tweets.shape

(90657, 14)

In [174]:
#add usernames to all_tweets dataframe
all_user.rename(columns={"id": "author_id"}, inplace=True)
all_tweets = pd.merge(all_tweets,all_user[['author_id','username']],on='author_id', how='inner')
all_tweets = all_tweets.drop_duplicates('id')
all_tweets.shape

(90657, 15)

### Save Files

In [175]:
#Option 1: Save data as separate files

# save dataframe to pickle
all_user.to_pickle("users.pkl")
all_tweets.to_pickle("tweets.pkl")
all_ref_tweets.to_pickle("reftweets.pkl")
all_places.to_pickle("places.pkl")
all_media.to_pickle("media.pkl")


In [163]:
print('Number of tweets fetched (rows, columns):', all_tweets.shape)
print('Number of users, both authors AND mentioned users (rows, columns):', all_user.shape)
print('Number of ref_tweets (rows, columns):', all_ref_tweets.shape)
print('Number of places, tagged by users in tweets (rows, columns):', all_places.shape)
print('Number of media in tweets (rows, columns):', all_media.shape)

Number of tweets fetched (rows, columns): (1044, 14)
Number of users, both authors AND mentioned users (rows, columns): (3337, 16)
Number of ref_tweets (rows, columns): (590, 13)
Number of places, tagged by users in tweets (rows, columns): (30, 7)
Number of media in tweets (rows, columns): (343, 7)


In [118]:
#---user---#
#authors and mentioned users

#---tweets---#
#tweets that are either from a specified account or include a specfified hashtag

#---ref_tweets---#
#A list of Tweets this Tweet refers to. For example, if the parent Tweet is a Retweet,
#a Retweet with comment (also known as Quoted Tweet) or a Reply, it will include the related
#Tweet referenced to by its parent.

#---places---#
#When including the expansions=geo.place_id parameter, this includes a list of referenced places
#in Tweets, if tagged by a user

#---media---#
#When including the expansions=attachments.media_keys parameter, this includes a list of images,
#videos, and GIFs included in Tweets in the form of media objects 

### Retrieve Tweet Images and Save

In [ ]:
import urllib.request

In [79]:
#load
media_file = pd.read_pickle('media.pkl')
media_file.shape

(365309, 9)

In [80]:
#remove duplicates
media_file.drop_duplicates(subset='media_key', inplace=True)

In [120]:
#shuffle dataframe to get images from random point in time
media_file = media_file.sample(frac = 1)

In [124]:
#save files, retrieved from url
def retrieve_imagefiles(limit):
    count = 0
    for index, row in media_file.iterrows():
            if count > limit:
                break
            try:
                urllib.request.urlretrieve(row['url'], f"data/vision_input/{row['media_key']}.jpg")
                print(row['index'], row['media_key'], row['url'])
            except:
                print('passed')
                pass
                
            count +=1

In [125]:
#specify number of images that will be saved locally
retrieve_imagefiles(10000)

99073 3_1238356776971579392 https://pbs.twimg.com/media/ES-G7XSWsAAIu75.jpg
268164 3_1110885655310188544 https://pbs.twimg.com/media/D2qonibXQAAPgp0.jpg
150462 3_1177673407145508864 https://pbs.twimg.com/media/EFfvuWZVAAAkTRy.jpg
23304 3_1120411241204338692 https://pbs.twimg.com/media/D4yAFnJW0AQl4pX.jpg
275960 3_1106746712784281600 https://pbs.twimg.com/media/D1v0RX_WoAAUlmw.jpg
113654 3_1211590687931715584 https://pbs.twimg.com/media/ENBvT7dXUAAA6Mt.jpg
36013 3_1441301238151221251 https://pbs.twimg.com/media/FACH1vpVkAMduWB.jpg
109823 3_1218278936834138113 https://pbs.twimg.com/media/EOgyPQNW4AE188s.jpg
131141 3_1195428686163513345 https://pbs.twimg.com/media/EJcEDnEWkAEkS8d.jpg
124628 3_1200412228689772544 https://pbs.twimg.com/media/EKi4j9sWwAAt4v5.jpg
220105 3_1148579017454510086 https://pbs.twimg.com/media/D_CSh-vX4AYOwvP.jpg
61532 3_1344107123265814528 https://pbs.twimg.com/media/Eqc6TqZUwAAkt5p.jpg
309382 3_1105367980102348800 https://pbs.twimg.com/media/D1cOUkgXcAAbxZ6.jpg
198

209499 3_1166422915874787333 https://pbs.twimg.com/media/EC_3dkaXkAUVzCP.jpg
176026 3_1175144317675069450 https://pbs.twimg.com/media/EE7zh97XYAowwtf.jpg
214979 3_1157112534878568449 https://pbs.twimg.com/media/EA7juEaWwAEIdaG.jpg
21898 3_1469315560324444162 https://pbs.twimg.com/media/FGQOt6QXsAIde52.jpg
66050 3_1322154827196043264 https://pbs.twimg.com/media/Elk8z20WkAAHAy8.jpg
6754 3_1511720500526989313 https://pbs.twimg.com/media/FPq1ySmWQAE5FAy.jpg
321215 3_1091676699887128576 https://pbs.twimg.com/media/DyZqLLJX0AAqyya.jpg
159365 3_1177540121769467905 https://pbs.twimg.com/media/EFd2gHrW4AERq5v.jpg
9398 3_1177706855616913409 https://pbs.twimg.com/media/EFgOJTqWoAEdKIw.jpg
191684 3_1175018050338377728 https://pbs.twimg.com/media/EE6AsPcXUAAJDOm.jpg
46650 3_1402334511157891081 https://pbs.twimg.com/media/E3YXztnWUAkPXrW.jpg
217351 3_1152220296662933506 https://pbs.twimg.com/media/D_2CQU2W4AIzYa8.jpg
278773 3_1106652598290038791 https://pbs.twimg.com/media/D1uerMLXQAcRkJW.jpg
187673

165585 3_1176938644252352514 https://pbs.twimg.com/media/EFVTdhUXYAIot-w.jpg
129864 3_1197867586664439808 https://pbs.twimg.com/media/EJ-uOTcWoAAC4IU.jpg
80665 3_1290658368571478019 https://pbs.twimg.com/media/EelW8rEWkAMQQCv.jpg
276234 3_1106718905354080256 https://pbs.twimg.com/media/D1va-xPW0AAqxYU.jpg
48750 3_1392475882246316033 https://pbs.twimg.com/media/E1MRb--XIAEXlPa.jpg
86592 3_1268910381839183873 https://pbs.twimg.com/media/EZwTRPFUcAEpK-2.jpg
134471 3_1190186139178668033 https://pbs.twimg.com/media/EIRj_NpWkAEA5LG.jpg
112191 3_1214966971508412416 https://pbs.twimg.com/media/ENxuBf6XUAAJxTD.jpg
37253 3_1073669395560435712 https://pbs.twimg.com/media/DuZwoHCWkAAmqHk.jpg
16054 3_1174907661881491456 https://pbs.twimg.com/media/EE4cSyZUwAAPbgb.jpg
117491 3_1205489124968407043 https://pbs.twimg.com/media/ELrB-OJWoAM0wNA.jpg
7068 3_1510193773896159233 https://pbs.twimg.com/media/FPVJPGsWUAEX_hS.jpg
143509 3_1179493843034607617 https://pbs.twimg.com/media/EF5nZqjWkAEpRFn.jpg
208941

48186 3_1395046924298956802 https://pbs.twimg.com/media/E1wzyU_XMAIOfaC.png
190842 3_1175022738714636288 https://pbs.twimg.com/media/EE6E9JAXsAAE5T3.jpg
54240 3_1372852998586699777 https://pbs.twimg.com/media/Ew1ah02XEAEPS4I.jpg
150066 3_1177681566476242946 https://pbs.twimg.com/media/EFf3JSRXkAI6UaK.jpg
196325 3_1174981013732089856 https://pbs.twimg.com/media/EE5fAbVUYAAJGn9.jpg
304317 3_1106438758960902144 https://pbs.twimg.com/media/D1rcMGmWsAAC-H2.jpg
110160 3_1218176013168852994 https://pbs.twimg.com/media/EOfUoTrUwAII0gu.jpg
passed
39917 3_1428806433475334147 https://pbs.twimg.com/media/E9Qj4ZAXEAMeDdM.jpg
229722 3_1136874776079716352 https://pbs.twimg.com/media/D8b9ld5VUAAWQPw.jpg
133347 3_1192281898535587841 https://pbs.twimg.com/media/EIvWEdpXkAEoPzW.jpg
399 3_1507211742275080196 https://pbs.twimg.com/media/FOqxGBOakAQCrkW.jpg
59723 3_1352205728652660737 https://pbs.twimg.com/media/EsP_8ieXEAE7Y41.png
57585 3_1362100153520431110 https://pbs.twimg.com/media/Eucm33cXIAYJtBH.jpg


170420 3_1175401847458291714 https://pbs.twimg.com/media/EE_dwLIW4AIE_k6.jpg
82347 3_1284977476196880389 https://pbs.twimg.com/media/EdUoNN0X0AU0TKu.jpg
95820 3_1246029246083944448 https://pbs.twimg.com/media/EUrI_2CU0AAByxx.jpg
320821 3_1093478136208130055 https://pbs.twimg.com/media/DyzQkkZXgAcrwCG.jpg
133192 3_1192622340036689921 https://pbs.twimg.com/media/EI0LsxEVAAE--cK.jpg
264592 3_1113018516867813376 https://pbs.twimg.com/media/D3I8ca5WkAAMFAP.jpg
52859 3_1373248991719256064 https://pbs.twimg.com/media/Ew7CrqOW8AAah7Z.jpg
95893 3_1246002542821011456 https://pbs.twimg.com/media/EUqwtgoUcAAjqW_.jpg
passed
41551 3_1423115445796696064 https://pbs.twimg.com/media/E7_r9T0VcAAvpBK.png
24614 3_1110335639017410562 https://pbs.twimg.com/media/D2i0YYEW0AICLop.jpg
35081 3_1441372786123935747 https://pbs.twimg.com/media/FADI6YnVEAMRofH.jpg
234266 3_1132283184308215808 https://pbs.twimg.com/media/D7atjpnW4AAaueg.jpg
149604 3_1177690170822463488 https://pbs.twimg.com/media/EFf--H9XkAA4OMo.jpg

...

34451 3_1441399298227310592 https://pbs.twimg.com/media/FADhBl5WEAA4MRf.jpg
236237 3_1131989367856259075 https://pbs.twimg.com/media/D7WiVSCXYAMNeiV.jpg
132671 3_1192888183870148608 https://pbs.twimg.com/media/EI39e6hXUAAiZKG.jpg
15743 3_1174926415927529474 https://pbs.twimg.com/media/EE4tWaqVUAISUxl.jpg
228922 3_1137064229792571394 https://pbs.twimg.com/media/D8ep5H-W4AI1pjJ.jpg
15336 3_1174944825101041667 https://pbs.twimg.com/media/EE4-F-LUwAMx2RJ.jpg
111166 3_1216499241050365958 https://pbs.twimg.com/media/EOHfnUxW4AYscov.jpg
215997 3_1154709392677322752 https://pbs.twimg.com/media/EAZaEyFWwAAsnuX.jpg
57894 3_1360302626294935562 https://pbs.twimg.com/media/EuDECAtXMAoXIC-.png
100062 3_1235941767016525825 https://pbs.twimg.com/media/ESbyfSIWsAEZJCc.jpg
119142 3_1203860217433227264 https://pbs.twimg.com/media/ELT4fUtVAAAk3_G.jpg
159521 3_1177538407569993728 https://pbs.twimg.com/media/EFd08VyWsAA6LOF.jpg
184898 3_1175061134900117506 https://pbs.twimg.com/media/EE6n4F8WwAIygcj.jpg
249

94836 3_1249205821210230784 https://pbs.twimg.com/media/EVYSE2iWAAABXcL.jpg
312488 3_1101762974849687552 https://pbs.twimg.com/media/D0o_lpaX4AAoAJY.jpg
90297 3_1258418064653258752 https://pbs.twimg.com/media/EXbMj_qXkAAOi6O.jpg
26925 3_1455470846907191297 https://pbs.twimg.com/media/FDLfBhrXEAE-AZo.jpg
11990 3_1505981480882819081 https://pbs.twimg.com/media/FOZSLYZWUAkEIyI.jpg
66501 3_1319736618895495169 https://pbs.twimg.com/media/ElCldm5XUAEGsV7.jpg
156800 3_1177573865561755648 https://pbs.twimg.com/media/EFeVMRGXkAAK1Mr.jpg
110960 3_1217098407455666176 https://pbs.twimg.com/media/EOQAjZkWsAACVAD.jpg
128744 3_1199312491504906240 https://pbs.twimg.com/media/EKTQW1bWsAAMKLL.jpg
131622 3_1195257402980995072 https://pbs.twimg.com/media/EJZoRnfWkAArfbC.jpg
136387 3_1187601267826155520 https://pbs.twimg.com/media/EHs1D5eU4AAFIUq.jpg
267236 3_1111566048543862784 https://pbs.twimg.com/media/D20TbovXcAAfSYH.jpg
205665 3_1172043450927243270 https://pbs.twimg.com/media/EEPvTxHXYAYOP4_.jpg
1013

265786 3_1111709491177897984 https://pbs.twimg.com/media/D22V5GMWwAAiBb_.jpg
307185 3_1106139899772710912 https://pbs.twimg.com/media/D1nMYNXWsAA1aku.jpg
321240 3_1091630731695194112 https://pbs.twimg.com/media/DyZAXeSXgAA4HlZ.jpg
309191 3_1105436112896622593 https://pbs.twimg.com/media/D1dMSa8XgAEHD3c.jpg
214821 3_1157223543689699328 https://pbs.twimg.com/media/EA9IrodX4AAE6vQ.jpg
48524 3_1393153180918222850 https://pbs.twimg.com/media/E1V5b9JWEAIMBNu.jpg
100358 3_1235837154623811584 https://pbs.twimg.com/media/ESaTWCmU8AAHv2Y.jpg
passed
140464 3_1182546650109104128 https://pbs.twimg.com/media/EGk_6bIU0AAHl7-.jpg
183248 3_1175074228300931072 https://pbs.twimg.com/media/EE6zyOqX4AA8ifL.jpg
56282 3_1367639170202677252 https://pbs.twimg.com/media/EvrUlEjXMAQNpED.jpg
214322 3_1157818151536664581 https://pbs.twimg.com/media/EBFleXTXoAUREMV.jpg
250361 3_1124757065430380546 https://pbs.twimg.com/media/D5vwl3-WsAIG37U.jpg
35606 3_1096381728976003072 https://pbs.twimg.com/media/DzchX6XX4AAYUbV

299309 3_1106502260580716545 https://pbs.twimg.com/media/D1sV8YkUkAEL9W5.jpg
35614 3_1096381411362332674 https://pbs.twimg.com/media/DzchFbKX4AIQP19.jpg
214561 3_1157335264337027076 https://pbs.twimg.com/media/EA-uSoTXYAQ7kYV.jpg
39288 3_1432019871219453956 https://pbs.twimg.com/media/E9-OfFdWEAQfTm1.jpg
241915 3_1131859610074320896 https://pbs.twimg.com/media/D7UsUYoXkAAOYiz.jpg
140890 3_1182207543810121728 https://pbs.twimg.com/media/EGgLf1uW4AAsjw0.jpg
passed
317043 3_1097913389249110016 https://pbs.twimg.com/media/DzySaRhWoAAKC4A.jpg
4967 3_1517655852936704000 https://pbs.twimg.com/media/FQ_L9RWVsAA-v93.jpg
27167 3_1454906528947912709 https://pbs.twimg.com/media/FDDdx5dWQAUsOv9.jpg
269359 3_1109183138557767681 https://pbs.twimg.com/media/D2ScMBSWsAEyvoH.jpg
173036 3_1175203557512904705 https://pbs.twimg.com/media/EE8paLhVAAEs_Fk.jpg
60361 3_1348602871894466560 https://pbs.twimg.com/media/ErczKrJXYAAQoVX.jpg
7294 3_1217971618301075461 https://pbs.twimg.com/media/EOcau9dX4AUCup4.jpg


132026 3_1194497305287811073 https://pbs.twimg.com/media/EJO0-G7XYAECfAy.jpg
84638 3_1264940486382489600 https://pbs.twimg.com/media/EY34q6DWkAAR55J.jpg
101067 3_1233574259693441029 https://pbs.twimg.com/media/ER6JQOEX0AUoQj5.jpg
324861 3_1085954873332584448 https://pbs.twimg.com/media/DxIWNCMX0AALkNh.jpg
26834 3_1455715060983205888 https://pbs.twimg.com/media/FDO9IqAXEAA3OR3.jpg
102919 3_1230873451918917632 https://pbs.twimg.com/media/ERTw4hEWAAAEk_b.jpg
296324 3_1106518211229609984 https://pbs.twimg.com/media/D1skc1XXcAA0taH.jpg
292106 3_1106538497995956224 https://pbs.twimg.com/media/D1s25rdX4AAvOT4.jpg
304523 3_1106429817099042816 https://pbs.twimg.com/media/D1rUDnkWwAAfisZ.jpg
45108 3_1408816414732005381 https://pbs.twimg.com/media/E40fEKHXMAUBd3M.jpg
282429 3_1106607179174420485 https://pbs.twimg.com/media/D1t1XcyWoAUEftj.jpg
288338 3_1106559099175612417 https://pbs.twimg.com/media/D1tJo01WsAEMMF4.jpg
263965 3_1113842895864389632 https://pbs.twimg.com/media/D3UqNl-W4AAIenr.jpg
29

### Get Image Labels

In [126]:
from __future__ import print_function
import pandas as pd
from google.cloud import vision
import io
import os.path
import os



In [127]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/romanbreyer/Downloads/global-timer-355314-884e9e06301f.json"

In [128]:
def label_detection(path):
    """ Label detection function - returns Dataframe with results. Always returns 10 labels"""
    # Get image in correct encoding
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    # API Call
    client = vision.ImageAnnotatorClient()
    image = vision.Image(content = content)
    response = client.label_detection(image=image)  #Limit to 10 for consistency

    # Get file name
    file_name = os.path.basename(path)
    print("="*30)
    print(f"Label detection for {file_name}")
    # Create Dataframe for return - there are 10 labels return every time
    collect_label = pd.DataFrame()

    # Create helper list for Dataframe
    helper = [path, file_name]
    vertices_labels = {}
    # Iterate over all annotations and get labels with confidence score
    iter = 0
    for label in response.label_annotations:
        iter += 1
        #print(label.description, '(%.2f%%)' % (label.score*100.))
        helper.append(f"{label.description} ({round(label.score*100.,2)}%)")

    # check if there are 10 labels in list, otherwise fill with empty
    if len(helper) < 10:
        while len(helper) != 10:
            helper.append("no response")
    # Fill and return Dataframe
    collect_label[f"{file_name}"] = helper
    return collect_label





In [129]:
def emotion_detection(path):
    """ Face Detection function - returns DataFrame with results.  Possible Emotions: "joy, sorrow, anger, surprise")"""
    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    client = vision.ImageAnnotatorClient()
    image = vision.Image(content = content)
    response = client.face_detection(image=image)
    collect_emotions = pd.DataFrame()
    file_name = os.path.basename(path)
    print("="*30)
    print(f"Emotion detection for {file_name}")
    counter = 0
    for face in response.face_annotations:
        counter += 1
        likelihood_joy = vision.Likelihood(face.joy_likelihood)
        likelihood_anger = vision.Likelihood(face.anger_likelihood)
        likelihood_sorrow = vision.Likelihood(face.sorrow_likelihood)
        likelihood_surprise = vision.Likelihood(face.surprise_likelihood)
        vertices = ['(%s,%s)' % (v.x, v.y) for v in face.bounding_poly.vertices]

        collect_emotions[f"Face {counter} in file {file_name}"] = [path, file_name, str(vertices), likelihood_joy.name, likelihood_anger.name, likelihood_sorrow.name, likelihood_surprise.name]

    return collect_emotions



In [130]:
def emotion_pipeline(limit, dir, ignore_max = False):
    """Function to shoot X amount of files in directory through Vision API and aggregate results (limit before it gets costly should be 500)
        Iterates through the given directory and processes every picture until "num" is reached, calls emotion_detection() and label_detection() in every iteration (two API Calls)"""

    # Initialize Dataframes for export
    label_fetch = pd.DataFrame({"Row Name": ["FilePath","FileName","Label_1","Label_2","Label_3","Label_4","Label_5","Label_6","Label_7","Label_8","Label_9","Label_10"]})
    emotions_fetch = pd.DataFrame({"Category":["FilePath",'FileName','Bounds','JOY','ANGER','SORROW','SURPRISE']})

    print("=" * 30)
    # For Label and Face Detection, two separate Calls are needed. Account for this in the number of API Calls!!!
    if (not ignore_max) and limit > 490:
        print("Limit will exceed max. monthly API calls - try again with ignore_max = True")
        return emotions_fetch, label_fetch
    print(f"Getting Emotion and Label Recognition, for {limit} files in {dir}")



    #Set counter and iterate over files in dir until defined limit is reached
    counter = 0
    for file in os.listdir(dir):
        counter += 1
        if counter <= limit:
            filepath = os.path.join(dir, file)
            if os.path.isfile(filepath):
                df_emotions_single = emotion_detection(filepath)
                emotions_fetch = pd.concat([emotions_fetch,df_emotions_single], axis=1)

                df_labels_single = label_detection(filepath)
                label_fetch = pd.concat([label_fetch, df_labels_single], axis = 1)
        else:
            print("Limit reached, not all pictures processed")
            return emotions_fetch, label_fetch
    print("All pictures processed, limit was not reached")
    return emotions_fetch, label_fetch


    # Aggregate all results in DataFrame by joining

In [131]:
# Function to visualize results of a specific picture and the annotations made by Vision API
def get_jpeg():
    return

In [132]:
def get_path(path):
    """Check if Directory exists, if not -> create it. Then return the path."""

    if not path[-1:] == "\\": path = path + "\\"

    path_snippets = path[:-1].split("\\")
    path_check = ""

    for snip in path_snippets:
        path_check = f"{path_check}{snip}\\"
        if not os.path.exists(path_check): os.mkdir(path_check)

    return f"{path}"

In [133]:
def export_toCSV(df, csv_path, csv_name = "Export_data.csv", index = True, index_name = "Index", header = True, sep = ","):
    """ Handles export of dataframe /w some extras"""
    # Console Output
    print(f"\nExporting data to CSV({csv_path}{csv_name}")
    # Create Path if necessary
    csv_path = get_path(csv_path)
    # Export Data
    df.index.name = index_name
    if csv_name[-4:] != ".csv":
        csv_name += ".csv"
    df.to_csv(f"{csv_path}{csv_name}", sep = sep , index = index, header =header)

In [143]:
''' Sample function call'''
emotions, labels = emotion_pipeline(10000,"data/vision_input",ignore_max = True)



Getting Emotion and Label Recognition, for 10000 files in data/vision_input
Emotion detection for 3_1249341069394878464.jpg
Label detection for 3_1249341069394878464.jpg
Emotion detection for 3_1106516677045469185.jpg
Label detection for 3_1106516677045469185.jpg
Emotion detection for 3_1175016252043145217.jpg
Label detection for 3_1175016252043145217.jpg
Emotion detection for 3_1106511805814374400.jpg
Label detection for 3_1106511805814374400.jpg
Emotion detection for 3_1129372053692387330.jpg
Label detection for 3_1129372053692387330.jpg
Emotion detection for 3_1380395721405255680.jpg
Label detection for 3_1380395721405255680.jpg
Emotion detection for 3_1507404703562350594.jpg
Label detection for 3_1507404703562350594.jpg
Emotion detection for 3_1110496794063065088.jpg
Label detection for 3_1110496794063065088.jpg
Emotion detection for 3_1141915515700613120.jpg
Label detection for 3_1141915515700613120.jpg
Emotion detection for 3_1186950930257862661.jpg
Label detection for 3_11869509

Emotion detection for 3_1096438328881307648.jpg
Label detection for 3_1096438328881307648.jpg
Emotion detection for 3_1121642840453292032.jpg
Label detection for 3_1121642840453292032.jpg
Emotion detection for 3_1494639049914658818.jpg
Label detection for 3_1494639049914658818.jpg
Emotion detection for 3_1309650549760032768.jpg
Label detection for 3_1309650549760032768.jpg
Emotion detection for 3_1106763142867107841.jpg
Label detection for 3_1106763142867107841.jpg
Emotion detection for 3_1086339746660536320.jpg
Label detection for 3_1086339746660536320.jpg
Emotion detection for 3_1218224832367624192.jpg
Label detection for 3_1218224832367624192.jpg
Emotion detection for 3_1450574018042466305.jpg
Label detection for 3_1450574018042466305.jpg
Emotion detection for 3_1049385622950031362.jpg
Label detection for 3_1049385622950031362.jpg
Emotion detection for 3_1177561836222394368.jpg
Label detection for 3_1177561836222394368.jpg
Emotion detection for 3_1094291742508138503.jpg
Label detect

Emotion detection for 3_1106451606252515328.jpg
Label detection for 3_1106451606252515328.jpg
Emotion detection for 3_1200347646914895872.jpg
Label detection for 3_1200347646914895872.jpg
Emotion detection for 3_1362916913513332738.jpg
Label detection for 3_1362916913513332738.jpg
Emotion detection for 3_1106544548040327168.jpg
Label detection for 3_1106544548040327168.jpg
Emotion detection for 3_1515992906787692548.jpg
Label detection for 3_1515992906787692548.jpg
Emotion detection for 3_1279301815582855173.jpg
Label detection for 3_1279301815582855173.jpg
Emotion detection for 3_1106334496134094848.jpg
Label detection for 3_1106334496134094848.jpg
Emotion detection for 3_1177570536618168320.jpg
Label detection for 3_1177570536618168320.jpg
Emotion detection for 3_1372849050727419906.jpg
Label detection for 3_1372849050727419906.jpg
Emotion detection for 3_1153569235668492290.jpg
Label detection for 3_1153569235668492290.jpg
Emotion detection for 3_1232704594117087232.jpg
Label detect

Emotion detection for 3_1149699049064452097.jpg
Label detection for 3_1149699049064452097.jpg
Emotion detection for 3_1175132647716208641.jpg
Label detection for 3_1175132647716208641.jpg
Emotion detection for 3_1134712604713074690.jpg
Label detection for 3_1134712604713074690.jpg
Emotion detection for 3_1179435479021826053.jpg
Label detection for 3_1179435479021826053.jpg
Emotion detection for 3_1175092966244982784.jpg
Label detection for 3_1175092966244982784.jpg
Emotion detection for 3_1175091111699857409.jpg
Label detection for 3_1175091111699857409.jpg
Emotion detection for 3_1152926418222931971.jpg
Label detection for 3_1152926418222931971.jpg
Emotion detection for 3_1291394375952605184.jpg
Label detection for 3_1291394375952605184.jpg
Emotion detection for 3_1177664053407174656.jpg
Label detection for 3_1177664053407174656.jpg
Emotion detection for 3_1183027615923347456.jpg
Label detection for 3_1183027615923347456.jpg
Emotion detection for 3_1462079770196389893.jpg
Label detect

Emotion detection for 3_1096450600852692992.jpg
Label detection for 3_1096450600852692992.jpg
Emotion detection for 3_1174934055109836805.jpg
Label detection for 3_1174934055109836805.jpg
Emotion detection for 3_1365313215761178626.jpg
Label detection for 3_1365313215761178626.jpg
Emotion detection for 3_1106585916976627712.jpg
Label detection for 3_1106585916976627712.jpg
Emotion detection for 3_1177573865561755648.jpg
Label detection for 3_1177573865561755648.jpg
Emotion detection for 3_1106605908451307520.jpg
Label detection for 3_1106605908451307520.jpg
Emotion detection for 3_1129288741435129856.jpg
Label detection for 3_1129288741435129856.jpg
Emotion detection for 3_1106708791394004993.jpg
Label detection for 3_1106708791394004993.jpg
Emotion detection for 3_1175081388237840385.jpg
Label detection for 3_1175081388237840385.jpg
Emotion detection for 3_1129171806873563136.jpg
Label detection for 3_1129171806873563136.jpg
Emotion detection for 3_1177607487253618689.jpg
Label detect

...

Emotion detection for 3_1175069329999708162.jpg
Label detection for 3_1175069329999708162.jpg
Emotion detection for 3_1423104870362406916.jpg
Label detection for 3_1423104870362406916.jpg
Emotion detection for 3_1185635612654788610.jpg
Label detection for 3_1185635612654788610.jpg
Emotion detection for 3_1122841146491846656.jpg
Label detection for 3_1122841146491846656.jpg
Emotion detection for 3_1174761880889569280.jpg
Label detection for 3_1174761880889569280.jpg
Emotion detection for 3_1096127561397166081.jpg
Label detection for 3_1096127561397166081.jpg
Emotion detection for 3_1106519920647516160.jpg
Label detection for 3_1106519920647516160.jpg
Emotion detection for 3_1175020413597966336.jpg
Label detection for 3_1175020413597966336.jpg
Emotion detection for 3_1146534983592493057.jpg
Label detection for 3_1146534983592493057.jpg
Emotion detection for 3_1106544171802918912.jpg
Label detection for 3_1106544171802918912.jpg
Emotion detection for 3_1106569560730357761.jpg
Label detect

Emotion detection for 3_1105450642456408064.jpg
Label detection for 3_1105450642456408064.jpg
Emotion detection for 3_1106527036032589824.jpg
Label detection for 3_1106527036032589824.jpg
Emotion detection for 3_1108051918310518784.jpg
Label detection for 3_1108051918310518784.jpg
Emotion detection for 3_1174950820669882369.jpg
Label detection for 3_1174950820669882369.jpg
Emotion detection for 3_1501595423038914569.jpg
Label detection for 3_1501595423038914569.jpg
Emotion detection for 3_1131980215918440448.jpg
Label detection for 3_1131980215918440448.jpg
Emotion detection for 3_1103389760188542977.jpg
Label detection for 3_1103389760188542977.jpg
Emotion detection for 3_1129281481271062528.jpg
Label detection for 3_1129281481271062528.jpg
Emotion detection for 3_1286456815325024256.jpg
Label detection for 3_1286456815325024256.jpg
Emotion detection for 3_1119565992055713793.jpg
Label detection for 3_1119565992055713793.jpg
Emotion detection for 3_1096446941343420417.jpg
Label detect

Emotion detection for 3_1091327691939348480.jpg
Label detection for 3_1091327691939348480.jpg
Emotion detection for 3_1177687372307324928.jpg
Label detection for 3_1177687372307324928.jpg
Emotion detection for 3_1106706280729772033.jpg
Label detection for 3_1106706280729772033.jpg
Emotion detection for 3_1327007311987171329.jpg
Label detection for 3_1327007311987171329.jpg
Emotion detection for 3_1337271288344309761.jpg
Label detection for 3_1337271288344309761.jpg
Emotion detection for 3_1106488769488797696.jpg
Label detection for 3_1106488769488797696.jpg
Emotion detection for 3_1119283919134662662.jpg
Label detection for 3_1119283919134662662.jpg
Emotion detection for 3_1106553023390777345.jpg
Label detection for 3_1106553023390777345.jpg
Emotion detection for 3_1123122488722874368.jpg
Label detection for 3_1123122488722874368.jpg
Emotion detection for 3_1106486245344055296.jpg
Label detection for 3_1106486245344055296.jpg
Emotion detection for 3_1484924748597547014.jpg
Label detect

Emotion detection for 3_1106564428454133760.jpg
Label detection for 3_1106564428454133760.jpg
Emotion detection for 3_1096382587268931585.jpg
Label detection for 3_1096382587268931585.jpg
Emotion detection for 3_1106489158007209984.jpg
Label detection for 3_1106489158007209984.jpg
Emotion detection for 3_1203098422171049986.jpg
Label detection for 3_1203098422171049986.jpg
Emotion detection for 3_1156885976704331777.jpg
Label detection for 3_1156885976704331777.jpg
Emotion detection for 3_1333064069863989256.jpg
Label detection for 3_1333064069863989256.jpg
Emotion detection for 3_1096525013044350976.jpg
Label detection for 3_1096525013044350976.jpg
Emotion detection for 3_1161986873427992580.jpg
Label detection for 3_1161986873427992580.jpg
Emotion detection for 3_1391443803425345538.jpg
Label detection for 3_1391443803425345538.jpg
Emotion detection for 3_1205341581756096512.jpg
Label detection for 3_1205341581756096512.jpg
Emotion detection for 3_1177588257787449349.jpg
Label detect

Emotion detection for 3_1184765919370907649.jpg
Label detection for 3_1184765919370907649.jpg
Emotion detection for 3_1124245972920483840.jpg
Label detection for 3_1124245972920483840.jpg
Emotion detection for 3_1451547791227031558.jpg
Label detection for 3_1451547791227031558.jpg
Emotion detection for 3_1106517947646918657.jpg
Label detection for 3_1106517947646918657.jpg
Emotion detection for 3_1175116860565725184.jpg
Label detection for 3_1175116860565725184.jpg
Emotion detection for 3_1258418064653258752.jpg
Label detection for 3_1258418064653258752.jpg
Emotion detection for 3_1215341865681244161.jpg
Label detection for 3_1215341865681244161.jpg
Emotion detection for 3_1205411016215019520.jpg
Label detection for 3_1205411016215019520.jpg
Emotion detection for 3_1258766321694781442.jpg
Label detection for 3_1258766321694781442.jpg
Emotion detection for 3_1200492191187185675.jpg
Label detection for 3_1200492191187185675.jpg
Emotion detection for 3_1106863660893376512.jpg
Label detect

In [144]:
# Export File
export_toCSV(emotions,csv_path= "data/vision_output/", csv_name="Emotions_Analysis.csv" )


Exporting data to CSV(data/vision_output/Emotions_Analysis.csv


In [145]:
# Export File
export_toCSV(labels,csv_path= "data/vision_output/", csv_name="Labels_Analysis.csv" )


Exporting data to CSV(data/vision_output/Labels_Analysis.csv
